# **Ini Untuk EL SETUP**
run cellnya aja

In [1]:
import numpy as np
import os
import cv2 as cv
import mediapipe as mp
import math

# *Sesuain sama kelas yang mau dipake misalnya kursi roda wajib 5, soalnya kontrolnya 5 doang*

In [2]:
# DATA_PATH = os.path.join('My_Datasets')
DATA_PATH = os.path.join('El_Agung') #ganti ini bos, ini nama foldernya

actions = np.array(["Kanan", "Maju", "Stop", "Mundur", "Kiri" #kalau kursi roda ini udah fix, kalau yang lain sesuain
                    ])
no_sequences = 30
sequence_length = 30 #ini eksperiment aja kalau modellu jelek pake setting ini bisa ganti, boleh lebih banyak inget tergantung epoch dkk juga


In [3]:
def create_folder(actions, data_path, no_sequences):
    for action in actions: 
        for sequence in range(no_sequences):
            try: 
                os.makedirs(os.path.join(data_path, action, str(sequence)))
            except:
                pass


# **Collect Keypoint Values for Training and Testing**

In [4]:
#define mediapipe face mesh and drawing utils
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [5]:
#detect using mediapipe model
def media_pipe_detection(image, model):
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB) 
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv.cvtColor(image, cv.COLOR_RGB2BGR) 
    return image, results

#draw face landmarks
def draw_land_marks(image, results):
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            mp_drawing.draw_landmarks(
                image=image,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=1, circle_radius=1),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=1, circle_radius=1))

#extract keypoints from face mesh
def extract_keypoints(results):
    if results.multi_face_landmarks:
        keypoints = np.array([[res.x, res.y, res.z] for res in results.multi_face_landmarks[0].landmark]).flatten()
    else:
        keypoints = np.zeros(468*3)
    return keypoints

In [6]:
#capture video and collect data
cap = cv.VideoCapture(0)
with mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                ret, frame = cap.read()

                # Make detections
                image, results = media_pipe_detection(frame, face_mesh)
                
                # Show to screen
                if frame_num == 0: 
                    cv.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv.LINE_AA)
                    cv.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv.LINE_AA)
                    cv.imshow('OpenCV Feed', image)
                    cv.waitKey(5000)
                else:
                    cv.imshow('OpenCV Feed', image)

                # Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                if cv.waitKey(10) & 0xFF == ord('q'):
                    break

    cap.release()
    cv.destroyAllWindows()

# **Normalize Datasets**

In [7]:
coordinates = []

for action in actions:
     for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            for coor in res:
                coordinates.append(coor)


In [8]:
# Normalize coordinates
def normalization(coordinates):
    print("Koordinat awal:")
    print("Max: " + str(max(coordinates)))
    print("Min: " + str(min(coordinates)))

    coordinates = np.array(coordinates)
    coordinates -= np.mean(coordinates, axis=0)
    coordinates /= np.std(coordinates, axis=0)

    print("\nKoordinat normalisasi:")
    print("Max: " + str(max(coordinates)))
    print("Min: " + str(min(coordinates)))
    return coordinates

def save_normalization(paths, actions, no_sequences, sequence_length, coor_norm):
    total_landmarks = 468 * 3
    temp_index = 0

    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                saved_coor_norm = coor_norm[temp_index:temp_index+total_landmarks]

                norm_npy_path = os.path.join(paths, action, str(sequence), str(frame_num) + "-norm")
                np.save(norm_npy_path, saved_coor_norm)

                temp_index += total_landmarks


In [9]:
normalized_coor = normalization(coordinates)
save_normalization(DATA_PATH, actions, no_sequences, sequence_length, normalized_coor)